ENSEMBLE METHOD TO COMBINE YOLOV5 and UNET MODELS

LOAD YOLO REQUIREMENTS

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17752, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 17752 (delta 66), reused 36 (delta 36), pack-reused 17644 (from 4)
Receiving objects: 100% (17752/17752), 17.16 MiB | 16.38 MiB/s, done.
Resolving deltas: 100% (12051/12051), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 68.6 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings("ignore", message="`torch.cuda.amp.autocast")

MOUNT GOOGLE DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


LOADING YOLO V5 MODEL

In [ ]:
import torch
yolo_model = torch.hub.load('./', 'custom', path='/content/drive/MyDrive/MY_YOLO/MY_FINAL_YOLO_RUN_with_new_arch/weights/best.pt', source='local')
yolo_model.cuda()

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 v7.0-450-g781b9d57 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)

Fusing layers... 
YOLOv5x_new_arch summary: 434 layers, 117142446 parameters, 0 gradients, 267.7 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 80, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(80, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(160, 160, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(80, 80, kernel_size=(1, 1), stride=(1, 1

LOADING UNET MODEL

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
import cv2

class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.block(x)


class UNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.d1 = DoubleConv(3, 64);   self.p1 = nn.MaxPool2d(2)
        self.d2 = DoubleConv(64, 128); self.p2 = nn.MaxPool2d(2)
        self.d3 = DoubleConv(128, 256);self.p3 = nn.MaxPool2d(2)
        self.d4 = DoubleConv(256, 512);self.p4 = nn.MaxPool2d(2)

        self.b  = DoubleConv(512, 1024)

        self.u4 = nn.ConvTranspose2d(1024, 512, 2, 2); self.c4 = DoubleConv(1024, 512)
        self.u3 = nn.ConvTranspose2d(512, 256, 2, 2);  self.c3 = DoubleConv(512, 256)
        self.u2 = nn.ConvTranspose2d(256, 128, 2, 2);  self.c2 = DoubleConv(256, 128)
        self.u1 = nn.ConvTranspose2d(128, 64, 2, 2);   self.c1 = DoubleConv(128, 64)

        self.final = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        c1 = self.d1(x); p1 = self.p1(c1)
        c2 = self.d2(p1); p2 = self.p2(c2)
        c3 = self.d3(p2); p3 = self.p3(c3)
        c4 = self.d4(p3); p4 = self.p4(c4)

        b  = self.b(p4)

        u4 = self.u4(b); u4 = torch.cat([u4, c4], dim=1); c5 = self.c4(u4)
        u3 = self.u3(c5); u3 = torch.cat([u3, c3], dim=1); c6 = self.c3(u3)
        u2 = self.u2(c6); u2 = torch.cat([u2, c2], dim=1); c7 = self.c2(u2)
        u1 = self.u1(c7); u1 = torch.cat([u1, c1], dim=1); c8 = self.c1(u1)

        return self.final(c8)


# Load UNet weights
unet = UNet().to(device)
unet.load_state_dict(torch.load('/content/drive/MyDrive/kaggle_datasets/TuSimple/unet_lane_weights.pth', map_location=device))
unet.eval()


UNet(
  (d1): DoubleConv(
    (block): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (p1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (d2): DoubleConv(
    (block): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (p2

In [ ]:
import numpy as np

def unet_predict_conf(frame):
    """Return UNet lane confidence mask (0–1 float)."""
    h, w = frame.shape[:2]

    # Resize to model training size
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = cv2.resize(rgb, (512, 256))  # width=512, height=256

    # Convert to tensor + normalize
    t = T.ToTensor()(resized)
    t = T.Normalize([0.485, 0.456, 0.406],
                    [0.229, 0.224, 0.225])(t)
    t = t.unsqueeze(0).to(device)

    # Predict raw confidence (before threshold)
    with torch.no_grad():
        pred = torch.sigmoid(unet(t))[0, 0].cpu().numpy()

    # Resize back to original shape
    pred = cv2.resize(pred, (w, h))

    return pred  # float array between 0–1


def overlay_lane_clean(frame, conf):
    # Threshold strong lane responses
    mask = (conf > 0.15).astype(np.uint8)

    # Remove lane predictions from top half (ROI)
    h = mask.shape[0]
    mask[:h//2, :] = 0

    # Remove small noise near bottom
    mask = cv2.erode(mask, np.ones((3,3), np.uint8), iterations=1)

    # Skeletonization to thin the lanes
    skeleton = np.zeros_like(mask)
    temp = mask.copy()

    while True:
        eroded = cv2.erode(temp, np.ones((3,3), np.uint8))
        opened = cv2.morphologyEx(eroded, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
        subset = eroded - opened
        skeleton = cv2.bitwise_or(skeleton, subset)
        temp = eroded.copy()
        if cv2.countNonZero(temp) == 0:
            break

    # Thicken for visibility
    thick = cv2.dilate(skeleton, np.ones((3,3), np.uint8), iterations=2)

    # Apply green overlay
    overlay = frame.copy()
    overlay[thick == 1] = (0,255,0)

    # Blend result
    return cv2.addWeighted(frame, 0.8, overlay, 0.8, 0)

In [ ]:
import cv2
import numpy as np

video_in = "/content/drive/MyDrive/testing/10103534-hd_3840_2160_24fps.mp4"
video_out = "/content/drive/MyDrive/YOLO_UNET_ENSEMBLE_7.mp4"

cap = cv2.VideoCapture(video_in)

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(video_out, fourcc, 30.0,
                      (int(cap.get(3)), int(cap.get(4))))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    conf = unet_predict_conf(frame)
    lane_frame = overlay_lane_clean(frame, conf)

    yolo_results = yolo_model(lane_frame)
    final_frame = yolo_results.render()[0]

    out.write(final_frame)


cap.release()
out.release()

print("Saved:", video_out)


Saved: /content/drive/MyDrive/YOLO_UNET_ENSEMBLE_7.mp4
